In [26]:
print(f"* node_exporter via Prometheus **")

* node_exporter via Prometheus **


In [27]:
import json
import requests
import base64
import pandas as pd
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

In [28]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [29]:
''' Prometheus URL '''
host = "http://localhost:9091"

In [ ]:
env = 'DEV_ES_NODES'

In [30]:
ES_NODES_LIST =  "localhost"
# ES_NODES_LIST =  os.getenv(env)

In [31]:
''' https://restfulapi.net/http-status-codes/ '''
http_status_code = {
    200 : 'Indicates that the request has succeeded.',
    201 : 'Indicates that the request has succeeded and a new resource has been created as a result.',
    400 : 'The server could not understand the request due to incorrect syntax. The client should NOT repeat the request without modifications.',
    401 : 'Unauthorized rquests. Insufficient permissions',
    403 : 'Unauthorized request. Insufficient permissions. The client does not have access rights to the content. ',
    500 : 'The server encountered an unexpected condition that prevented it from fulfilling the request.'
}

In [32]:
''' base64 encode '''
def base64_encode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    encoded = '{}'.format(base64.b64encode(id_pass.encode('utf-8')).decode())
    # print(encoded)
    return encoded

In [33]:
''' base64 decode '''
def base64_decode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    return base64.b64decode(id_pass).decode('utf-8')

In [34]:
def get_headers(auth):
    ''' Elasticsearch Header '''
    ''' 
    Basic Authentication is a method for an HTTP user agent (e.g., a web browser) 
    to provide a username and password when making a request. 
    You can send the authorization header 
    when making requests and accessing to ES Cluster based on Search-Guard as X-pack. 
      
    Basic Auth : 
    {
        'Content-type': 'application/json', 
        'Authorization' : 'Basic base64.encode(id:password), 
        'Connection': 'close'
    }
    '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : 'Basic {}'.format(base64_encode_for_search_guard(auth)),
            'Connection': 'close'
    }

In [35]:
def request_to_prometheus(prometheus_query):
    try:
        ''' wxuser '''
        res = requests.get(
            url="{}/api/v1/query".format(host), 
            params=prometheus_query,
            headers=get_headers("es_view:123$qwer"), 
            verify=False)
    
        if not res.status_code == 200:
           print(res.status_code, http_status_code.get(res.status_code, "None"))
           print('\n')
           return
           
        # print(json.dumps(res.json(), indent=2))
        return res.json()
        # print('\n')
    except Exception as e:
        print(str(e))

In [36]:
get_headers("test:test")

{'Content-type': 'application/json',
 'Authorization': 'Basic dGVzdDp0ZXN0',
 'Connection': 'close'}

In [37]:
''' sample '''
''' https://docs.kanaries.net/ko/topics/Pandas/pandas-add-column '''
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 35, 40]
}
 
df = pd.DataFrame(data)
df.head(10)

,Name,Age
0,Alice,25
1,Bob,30
2,Charlie,35
3,David,40


In [38]:
df_dict = {}

In [39]:
# query = {
#     'query': 'basiccpuModelInfoGauge{server_job=~"localhost"}'
# }
query = {
    'query': """
basiccpuModelInfoGauge{server_job=~"%s"}
""" % (ES_NODES_LIST)
}

''' http://localhost:9090/api/v1/query?query=basiccpuModelInfoGauge{server_job=~"localhost|localhost1} '''
resp = request_to_prometheus(query)

In [40]:
_type, hostname, cpu_model, cpu_cores, cpu_logicalCnt, cpu_physicalCnt = [], [], [], [], [], []
ram_total, ram_used, ram_available, ram_used_percent = [], [], [], []

In [41]:
for each_dict in resp.get("data").get("result"):
    _type.append("cpu")
    cpu_model.append(each_dict.get("metric").get("cpu_model"))
    hostname.append(each_dict.get("metric").get("hostname"))
    cpu_cores.append(each_dict.get("metric").get("cpu_cores"))
    cpu_logicalCnt.append(each_dict.get("metric").get("cpu_logicalCnt"))
    cpu_physicalCnt.append(each_dict.get("metric").get("cpu_physicalCnt"))

    ram_total.append("-")
    ram_used.append("-")
    ram_available.append("-")
    ram_used_percent.append("-")
    
df_dict.update({"resource" : _type})
df_dict.update({"cpu_model" : cpu_model})
df_dict.update({"hostname" : hostname})
df_dict.update({"cpu_cores" : cpu_cores})
df_dict.update({"cpu_logicalCnt" : cpu_logicalCnt})
df_dict.update({"cpu_physicalCnt" : cpu_physicalCnt})

# print(json.dumps(df_dict, indent=2))        

In [42]:
env = 'Dev'

In [43]:
# query = {
#     'query': 'basicMemoryInfoGBGauge{server_job=~"localhost"}'
# }
query = {
    'query': """
basicMemoryInfoGBGauge{server_job=~"%s"}
""" % (ES_NODES_LIST)
}
resp = request_to_prometheus(query)

In [44]:
for each_dict in resp.get("data").get("result"):
    _type.append("memory")

    hostname.append(each_dict.get("metric").get("hostname"))

    cpu_model.append("-")
    cpu_cores.append("-")
    cpu_logicalCnt.append("-")
    cpu_physicalCnt.append("-")

    ram_total.append(each_dict.get("metric").get("ram_total"))
    ram_used.append(each_dict.get("metric").get("ram_used"))
    ram_available.append(each_dict.get("metric").get("ram_available"))
    ram_used_percent.append(each_dict.get("metric").get("ram_used_percent"))

df_dict.update({"resource" : _type})
df_dict.update({"hostname" : hostname})
df_dict.update({"ram_total" : ram_total})
df_dict.update({"ram_used" : ram_used})
df_dict.update({"ram_available" : ram_available})
df_dict.update({"ram_used_percent" : ram_used_percent})


In [45]:
# print(json.dumps(df_dict, indent=2)) 

In [46]:
df = pd.DataFrame.from_dict(df_dict)
# df.head(10)
display(df)

,resource,cpu_model,hostname,cpu_cores,cpu_logicalCnt,cpu_physicalCnt,ram_total,ram_used,ram_available,ram_used_percent


In [47]:
# Write DataFrame to CSV File with Default params.
filename = f"./[{env}]_df_resource_details.csv"
if not df.empty:
    df.to_csv(f"{filename}")
    print(f"{filename} was created..")